# Семинар 3. Наивное исправление опечаток

In [ ]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

In [ ]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

Посмотрите на пары предложений. В true лежат предл. без ошибок, в bad - с ошибками. Выведите любую пару

In [ ]:
# your code here 

In [ ]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = # разбейте предложение 1 в нижнем регистре по пробелам
    tokens_2 = # разбейте предложение 2 в нижнем регистре по пробелам
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

Передайте пару любых предложений из true и bad в функцию align_words, выведите на экран

In [ ]:
# your code here

Вытащим только неправильные варианты и заодно посчитаем долю ошибок.

In [ ]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [ ]:
# Посчитайте долю ошибок от общего кол-ва слов. Округлите ответ до сотых

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [ ]:
Counter(mistakes).most_common(10)

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Тексты из википедии для этого хорошо подходят.

In [ ]:
corpus = open('wiki_data.txt', encoding='utf8').read()

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [ ]:
# создаем словарь
vocab = set(re.findall('\w+', corpus.lower()))


Посмотрите, сколько слов всего в нашем словаре по википедии

In [ ]:
# your code here

In [ ]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0 
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка 
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

def predict_mistaken(word, vocab):
    '''проверяем, есть ли слово, не совпавшее с true, в словаре.'''
    if word in vocab:
        return 0
    return 1


for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))
    

### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига.

Ошибки вычисляются также, по словарю. Но ещё мы собираем частотности, чтобы потом выбирать самое вероятное исправление.

In [ ]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [ ]:
WORDS.most_common(10)

Потестируйте наш словарь. Найдите какое-то реальное слово, которого там не будет. Это будет значить, что наш словарь может провоцировать ошибки

In [ ]:
# your code here

In [ ]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())

def P(word): 
    '''Вычисляем вероятность слова'''
    
    return WORDS[word] / N


In [ ]:
P('в')

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [ ]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
%%time
correction('сонце')

In [ ]:
%%time
correction('опофеоз')

### Посмотрим на все возможные варианты удаления, замены, вставки и перестановки

In [ ]:
word = 'сонце'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

### Удаление

In [ ]:
deletes = [L + R[1:] for L, R in splits if R]

In [ ]:
deletes[:10]

### Перестановка соседних

In [ ]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [ ]:
transposes[:10]

### Замена

In [ ]:
deletes = [L + R[1:] for L, R in splits if R]
len(replaces)

### Вставка

In [ ]:
inserts = [L + c + R for L, R in splits for c in letters]

In [ ]:
inserts[:10]

In [ ]:
len(inserts)

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
                
print('DONE!')
        

In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

Выведите долю ошибок. Как изменилось по сравнению с началом?

In [ ]:
# your code here

Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [ ]:
%%time
correction('солнце')

In [ ]:
%%time
correction('насмехатьсяаававттававаываываы')

Посмотрим, как исправляются самые частотные ошибки.

In [ ]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]